# Time-independent ODE

$$
\frac{\mathrm{d} u(t)}{\mathrm{d} t} = -A u(t) + b(t),\quad u(0) = \vec{u}_0
$$

References

[1] Quantum algorithm for linear non-unitary dynamics with near-optimal dependence on all parameters https://arxiv.org/pdf/2312.03916

In [6]:
import numpy as np
import numpy.linalg as nl
import scipy
import scipy.linalg as sl
import scipy.integrate as spi
# import scipy.sparse as sp
# import scipy.sparse.linalg as ssl

## Define Parameters

We also define inhomogeneous term $b(t)$ and its first derivative as functions.

In [7]:
# def bt(t):
#     return sp.csc_matrix(np.array([ 2*t, 1j*t, 0, t ])).reshape(-1,1)
def bt(t):
    # return np.array([ 2*t, t, 0, 0.5*t ]).reshape(-1,1)
    return np.array([ 1*t+1j, 1, 3j*t, (2+1j)*t, 0.3*t, -2j, -1j*t, 0.5 ],dtype=complex).reshape(-1,1) ## dtype complex is necessary for scipy to compute complex integral

def bt_d1(t):
    ## First derivative of b(t), for computing Xi
    ## It may not be useful for first-degree polynomial b(t) with t from 0 to 1 anyway
    return np.array([ 1, 0, 3j, 2+1j, 0.3, 0, -1j, 0 ],dtype=complex).reshape(-1,1) ## dtype complex is necessary for scipy to compute complex integral

rng = np.random.Generator(np.random.PCG64(15817 ))

## Random matrix
dim = 8
A = np.matrix(rng.random((dim,dim)),dtype=complex) + 2*np.identity(dim) + 1j*np.matrix(rng.random((dim,dim)),dtype=complex) 
## Random initial state
u0 = np.matrix( rng.random((dim,1)) ,dtype=complex) ## dtype complex is necessary for scipy to compute complex integral
u0 = u0/nl.norm(u0, ord=2)

T = 1
beta = 0.9 # 0< beta < 1
epsilon = 1e-2

A, u0

(matrix([[2.19360455+0.32035373j, 0.64607675+0.81410473j,
          0.89783464+0.80453225j, 0.59698031+0.69797902j,
          0.42620768+0.26497782j, 0.9836633 +0.46534977j,
          0.81174259+0.56811647j, 0.44175126+0.56775619j],
         [0.88850987+0.0206711j , 2.24125337+0.18470654j,
          0.24905585+0.88453324j, 0.10732185+0.79513592j,
          0.60971287+0.03673279j, 0.09969004+0.61911263j,
          0.21526444+0.61681828j, 0.59802133+0.43174139j],
         [0.85081429+0.0354456j , 0.29551618+0.90532471j,
          2.05769736+0.62978209j, 0.78290271+0.65780738j,
          0.74724372+0.86280139j, 0.09471192+0.95623527j,
          0.07658047+0.02300761j, 0.03886701+0.1706145j ],
         [0.78725106+0.96337034j, 0.69414311+0.61139016j,
          0.90245452+0.62965932j, 2.39752611+0.75308156j,
          0.38033069+0.02415316j, 0.7227812 +0.96266081j,
          0.97241671+0.64976287j, 0.05715367+0.26412515j],
         [0.91376763+0.67574559j, 0.67057342+0.57724607j,
          

## Define $L$ and $H$ from $A = L + iH$

Need to make sure $L$ is a PSD matrix.

In [8]:
def generate_LH(A):
    ## Return L and H
    return 0.5*(A+A.conjugate().transpose()), -0.5j*(A-A.conjugate().transpose())

L_exp,H_exp = generate_LH(A)
L_eigvals = nl.eigh(L_exp)[0]

for eigval in L_eigvals:
    if eigval < 0:
        raise Exception("L has negative eigenvalues:", eigval)

###
print("||L+iH - A||=",nl.norm(L_exp+1j*H_exp - A, ord=2))
print("||L - L^†||=", nl.norm(L_exp-L_exp.conjugate().transpose(), ord=2))
print("||H - H^†||=", nl.norm(H_exp-H_exp.conjugate().transpose(), ord=2))
print("L eigenvalues:", L_eigvals)

L_exp, H_exp
# ssl.eigsh(L, k=1, which='SA')[0]
# L eigenvalues: [0.24207954 1.21032208 1.52765635 3.04428706]

||L+iH - A||= 2.501082684956281e-16
||L - L^†||= 0.0
||H - H^†||= 0.0
L eigenvalues: [0.3854782  1.09763634 1.55089533 1.76463362 2.22288869 2.61353476
 2.76987978 6.11735584]


(matrix([[2.19360455+0.j        , 0.76729331+0.39671682j,
          0.87432447+0.38454333j, 0.69211569-0.13269566j,
          0.66998766-0.20538389j, 0.65413085-0.19832715j,
          0.76451897+0.14737722j, 0.63215737+0.0319203j ],
         [0.76729331-0.39671682j, 2.24125337+0.j        ,
          0.27228601-0.01039573j, 0.40073248+0.09187288j,
          0.64014315-0.27025664j, 0.07869171-0.1297514j ,
          0.30518911-0.05816483j, 0.39247888+0.10230295j],
         [0.87432447-0.38454333j, 0.27228601+0.01039573j,
          2.05769736+0.j        , 0.84267861+0.01407403j,
          0.64437573+0.40157411j, 0.42023951+0.14710078j,
          0.22920059-0.39934548j, 0.3171227 -0.09756498j],
         [0.69211569+0.13269566j, 0.40073248-0.09187288j,
          0.84267861-0.01407403j, 2.39752611+0.j        ,
          0.24914171-0.32523486j, 0.40481201+0.0930884j ,
          0.8003652 +0.32047204j, 0.2878594 -0.05730907j],
         [0.66998766+0.20538389j, 0.64014315+0.27025664j,
          

## Homegenous Part


Solve time-independet homogeneous ODE
$$
\frac{\mathrm{d} u(t)}{\mathrm{d} t} = -A u(t),\quad u(0) = u_0
$$

## Define Kernal

In [9]:
def cbeta(beta):
    return 2*np.pi*np.exp(-2**beta)

def kernel_func(beta, z, cb_func=cbeta):
    ## 0 < beta < 1
    ## Kernal functon Eq. (7) in [1]
    return 1/(cb_func(beta)*np.exp( (1+1j*z)**beta ) )

def gk(beta, k, cb_func=cbeta):
    ## 0 < beta < 1
    ## Coefficient f(k)/(1-ik) in Eq. (6) in [1], or g(k) in Eq. (60) in [1]
    return kernel_func(beta, k, cb_func=cb_func)/(1- 1j*k)

kernel_func(beta, 2), gk(beta, 2)

((-0.05385634126278436-0.3309448346013612j),
 (0.1216066655879876-0.08773150342538599j))

## Define Time-Independent Unitary Time-Evolution Operator $U(T, s, k) = \mathcal{T} e^{-i \int^T_{s} (kL+H) \mathrm{d} s'}= \mathcal{T} e^{-i (T-s) (kL+H)}$

(This should be a quantum step)

In [10]:
def utsk(tT, s, k, L, H):
    ## U(T,s,k) in Eq. (70) in [1]
    ## ignore time-ordering operator since we don't trotter it
    return sl.expm(-1j*(tT-s)*(k*L+H))

def utk(tT, k, L, H):
    # special case when s=0
    return utsk(tT, 0, k, L, H)

utk(T, 1, L_exp, H_exp)

array([[ 0.23152028-0.37875133j, -0.13823678-0.15766513j,
        -0.10023735-0.0609193j , -0.34440963+0.28071025j,
        -0.16991313-0.08549655j, -0.25355608+0.43456499j,
        -0.07577076+0.38700836j, -0.20436629+0.24739311j],
       [-0.16098259+0.50203629j,  0.01049594-0.62513358j,
        -0.27657565+0.16647882j, -0.14789817-0.09463842j,
        -0.13012865+0.29142326j,  0.03619343+0.12445107j,
        -0.03589648+0.07993383j,  0.1763681 +0.19662077j],
       [-0.23443991+0.38402538j, -0.16156223+0.32329664j,
        -0.11740195-0.27835801j, -0.31681646+0.30869362j,
        -0.15789126-0.12293725j, -0.11811549-0.40926817j,
         0.12081675+0.36882468j,  0.08870403-0.00410923j],
       [-0.19341516-0.01079809j, -0.02550462+0.49447791j,
        -0.26665835+0.13937673j, -0.27681149-0.41093379j,
         0.2772666 +0.33314268j, -0.20175954+0.3457848j ,
        -0.0221147 +0.09687115j,  0.11756566-0.09727147j],
       [-0.10669929+0.1611931j , -0.25013906+0.15106095j,
        -0

## Discretization Parameters

In [11]:
## See Eq.(63) to (66) in [1]

def trunc_K(beta, epsilon, h1):
    ## Truncate the integral \int_\mathbb{R} g(k)U(T,k)dk to \int_{-K}^K g(k)U(T,k)dk
    ## (63) in [1]
    temp = (np.log(1/epsilon))**(1/beta)
    return np.ceil(temp/h1)*h1

def trunc_Kexact(beta, epsilon, h1):
    ## Truncate the integral \int_\mathbb{R} g(k)U(T,k)dk to \int_{-K}^K g(k)U(T,k)dk
    ## (63) in [1]
    temp_B = int(np.ceil(1/beta))
    temp_1 = 2**(temp_B+1)*np.prod(np.arange(1,temp_B+1))/( cbeta(beta)*(np.cos(beta*np.pi*0.5)**temp_B) )
    temp_2 = (np.log(temp_1)+np.log(1/epsilon))/(np.cos(beta*np.pi*0.5)) ## should also - ln(K), but maybe too small to care
    temp = (2*temp_2)**(1/beta)
    return np.ceil(temp/h1)*h1

def step_size_h1(T, L):
    ## step size h1 to discretize \int_{-K}^K g(k)U(T,k)dk
    ## since L is time-independent, ||L|| = max_t ||L(t)||
    ## Eq. (65) in [1]
    return 1/(np.exp(1)*T*nl.norm(L, ord=2))

def n_node_Q(beta, epsilon, rangeK, cb_func=cbeta):
    ## Number of nodes in each shorter interval [mh_1, (m+1)h_1] for m = -K/h1 to K/h1
    ## Eq. (65) in [1]
    temp = 1/np.log(4) * np.log( (8/(3*cb_func(beta))) * (rangeK/epsilon) )
    return int(np.ceil(temp))

## Gaussian-Legendre quadrature
def gauss_quadrature(lower_end,interval_step,num_slices):
    ## obtain the standard Sampling points and weights 
    ## for Gauss-Legendre quadrature in [lower_end, lower_end+interval_step]
    ##
    ## Convert C_i, X_i in range [-1,1] to c_i, x_i in range [a+mh,a+mh+h], 
    ## the convertion is c_i = h/2*C_i and x_i = h/2 x_i + (a+mh+a+mh+h)/2 = h/2 x_i + (a + mh) +h/2
    ##
    samp_points, weights = np.polynomial.legendre.leggauss(num_slices)
    ## Change of variable to [a,b], see A.4 in [1]
    weights_ab = 0.5*interval_step*weights
    samp_points_ab = 0.5*interval_step*samp_points + lower_end + 0.5*interval_step
    return samp_points_ab, weights_ab ## kqm and wq in Eq. (61) in [1]



h1_exp = step_size_h1(T, L_exp)
K_exp = trunc_K(beta, epsilon, h1_exp)
Q_exp = n_node_Q(beta, epsilon, K_exp)
M_exp = int(2*K_exp*Q_exp/h1_exp)
m_exp = 10
points_exp, weights_exp = gauss_quadrature(m_exp*h1_exp, h1_exp, Q_exp)

print("h1_exp:", h1_exp)
print("K_exp:", K_exp)
print("Q_exp:", Q_exp)
print("M_exp:", M_exp)
print("kqm with m={m_exp}:", points_exp)
print("wq:", weights_exp)

h1_exp: 0.060137002132342826
K_exp: 5.4724671940431975
Q_exp: 6
M_exp: 1092
kqm with m={m_exp}: [0.60340056 0.61155695 0.6242636  0.63861344 0.6513201  0.65947648]
wq: [0.00515147 0.01084756 0.01406947 0.01406947 0.01084756 0.00515147]


## Summarize the Computation for Homogeneous Solution $\sum_{m=-K/h_1}^{K/h_1 -1}\sum_{q=0}^{Q-1}w_qg(k_{q,m}) U(T,k_{q,m})$

The summation should use LCU (quantum step)

In [12]:
## Classical LCHS method for time-independent Hamiltonian homogeneous ODE
def class_LCHS_tihs(A, u0, tT, beta, epsilon, verbose=False): 
    L,H = generate_LH(A)
    h1 = step_size_h1(tT, L)  ## step size h1 for [-K, K], (65) in [1]
    K = 2*trunc_K(beta, epsilon, h1) ## integral range -K to K, (63) in [1] ## "2" here is just for higher accuracy since formula is big-O
    Q = n_node_Q(beta, epsilon, K)  ## number of nodes in each subinterval [mh_1, (m+1)h_1] for m = -K/h1 to K/h1, (65) in [1]
    ##
    kh1 = int(K/h1)
    M = int(2*kh1*Q) ## number of nodes in total, (65) in [1] ## just for checking
    ##
    res_mat = np.zeros(A.shape, dtype=complex)
    c_sum = 0 ## compute ||c||_1
    for munshit in range(2*kh1):
        m = -kh1+munshit ## shift to start from -K/h1
        kqms, wqs = gauss_quadrature(m*h1, h1, Q) ## Gaussian quadrature points and weights in [mh_1, (m+1)h_1]
        
        for qi in range(Q):
            cqm = wqs[qi]*gk(beta, kqms[qi], cb_func=cbeta)
            c_sum += np.abs(cqm)
            res_mat += cqm* utk(tT, kqms[qi], L, H)  ## w_q*g(k_{q,m})*U(T, k_{q,m}) ## (61) in [1] (This should be quantum)
    if verbose:
        print("Preset parameters T = ", tT, "beta = ", beta, "epsilon = ", epsilon)
        print("Truncation range [-K,K] K =", K)
        print("Step size h1 =", h1)
        print("Number of nodes in [mh_1, (m+1)h_1] Q =", Q)
        print("Total number of nodes M =", M)
        print("||c||_1 =", c_sum)
    u0_norm = u0/nl.norm(u0,ord=2)
    uT = res_mat.dot(u0_norm)
    return res_mat, uT

exp_op, uT = class_LCHS_tihs(A, u0, T, beta, epsilon, verbose=True)
uT = np.array(uT).reshape(-1)
uT_norm = uT/nl.norm(uT,ord=2)
print("u(T)=", uT)
print("u(T)/||u(T)||=", uT_norm)

Preset parameters T =  1 beta =  0.9 epsilon =  0.01
Truncation range [-K,K] K = 10.944934388086395
Step size h1 = 0.060137002132342826
Number of nodes in [mh_1, (m+1)h_1] Q = 6
Total number of nodes M = 2184
||c||_1 = 1.8657907150364312
u(T)= [-0.05605665+0.00883657j -0.02036921-0.0526817j   0.09148876-0.00032831j
 -0.061482  +0.04958058j  0.00180499+0.06495725j -0.0464175 -0.03942093j
  0.05268995-0.02966901j  0.01956579-0.00648528j]
u(T)/||u(T)||= [-0.30843255+0.0486202j  -0.11207459-0.28986305j  0.50338561-0.00180639j
 -0.33828365+0.27280015j  0.00993133+0.35740505j -0.2553964 -0.2169002j
  0.28990843-0.16324356j  0.10765404-0.03568303j]


## Compare with Analytic Result

For simple ODE like
$$
\frac{\mathrm{d} u(t)}{\mathrm{d} t} = -A u(t),\quad u(0) = \vec{u}_0
$$
Let $\lambda_i$ and $\vec{v}_i$ be $i^{th}$ eigenvalue and eigenvectors of $-A$, then we have solution
$$
u(T) = \sum_i c_i e^{\lambda_i T} \vec{v}_i
$$
where $c_i$ is solved from the linear system $\sum_i c_i \vec{v}_i = \vec{u}_0 \rightarrow  V\vec{c} = \vec{u}_0$ and $V$ is the matrix formed by eigenvectors $\vec{v}_i$'s.


In [13]:
eigval_A, eigvec_A = sl.eig(-A)
sol_coeffs = sl.solve(eigvec_A, u0).flatten()
analytic_sol = np.zeros(u0.shape[0], dtype=complex)
for i in range(u0.shape[0]):
    analytic_sol += sol_coeffs[i]*np.exp(eigval_A[i]*T)*eigvec_A[:,i]

analytic_sol = np.array(analytic_sol).flatten()
analytic_sol_norm = analytic_sol/nl.norm(analytic_sol,ord=2)

print("epsilon:", epsilon)
print("\nUn-normalized")
print("Homogeneous solution from Analytic:", analytic_sol, "Norm=", nl.norm(analytic_sol,ord=2))
print("Homogeneous solution from LCHS    :", uT, "Norm=", nl.norm(uT,ord=2))
print("Homogeneous solution error:", nl.norm(uT - analytic_sol,ord=2))
print("\nNormalized")
print("Homogeneous solution from Analytic:", analytic_sol_norm, "Norm=", nl.norm(analytic_sol_norm,ord=2))
print("Homogeneous solution from LCHS    :", uT_norm, "Norm=", nl.norm(uT_norm,ord=2))
print("Homogeneous solution error:", nl.norm(uT_norm - analytic_sol_norm,ord=2))

epsilon: 0.01

Un-normalized
Homogeneous solution from Analytic: [-0.05956616+0.00788714j -0.02081097-0.05219635j  0.09419924+0.0004033j
 -0.06461458+0.0508799j   0.00017558+0.06737744j -0.04733338-0.03858971j
  0.05417277-0.02991086j  0.0188974 -0.00596751j] Norm= 0.18680683202452314
Homogeneous solution from LCHS    : [-0.05605665+0.00883657j -0.02036921-0.0526817j   0.09148876-0.00032831j
 -0.061482  +0.04958058j  0.00180499+0.06495725j -0.0464175 -0.03942093j
  0.05268995-0.02966901j  0.01956579-0.00648528j] Norm= 0.18174687981381626
Homogeneous solution error: 0.0067857243962616606

Normalized
Homogeneous solution from Analytic: [-0.31886497+0.04222083j -0.11140371-0.27941348j  0.50426013+0.0021589j
 -0.3458898 +0.27236636j  0.00093991+0.36067976j -0.25338143-0.20657549j
  0.2899935 -0.16011652j  0.10116013-0.03194479j] Norm= 0.9999999999999999
Homogeneous solution from LCHS    : [-0.30843255+0.0486202j  -0.11207459-0.28986305j  0.50338561-0.00180639j
 -0.33828365+0.27280015j  0.0

### Compare with Scipy

In [14]:
def ode_func_ho(t,u):
    return np.array(-A.dot(u).reshape(-1))[0]

spi_sol_ho = spi.solve_ivp(ode_func_ho, [0,T],np.array(u0.reshape(-1))[0], method='RK45')
spi_uT_ho = spi_sol_ho.y[:,-1]
spi_uT_ho_norm = spi_uT_ho/nl.norm(spi_uT_ho,ord=2)

if nl.norm(uT.imag,ord=2) < 1e-12:
    uT = uT.real

if nl.norm(uT_norm.imag,ord=2) < 1e-12:
    uT_norm = uT_norm.real

uT_err = nl.norm(uT - spi_uT_ho,ord=2)
uT_err_norm = nl.norm(uT_norm - spi_uT_ho_norm,ord=2)

print("epsilon:", epsilon)
print("\nUn-normalized")
print("Homogeneous solution from scipy:", spi_uT_ho, "Norm=", nl.norm(spi_uT_ho,ord=2))
print("Homogeneous solution from LCHS :", uT, "Norm=", nl.norm(uT,ord=2))
print("Homogeneous solution error:", uT_err, "Relative error:", uT_err/nl.norm(spi_uT_ho,ord=2))
print("\nNormalized")
print("Homogeneous solution from scipy:", spi_uT_ho_norm, "Norm=", nl.norm(spi_uT_ho_norm,ord=2))
print("Homogeneous solution from LCHS :", uT_norm, "Norm=", nl.norm(uT_norm,ord=2))
print("Homogeneous solution error:", uT_err_norm, "Relative error:", uT_err_norm/nl.norm(spi_uT_ho_norm,ord=2))

epsilon: 0.01

Un-normalized
Homogeneous solution from scipy: [-0.05955654+0.00789104j -0.02080481-0.05219285j  0.09420551+0.00040743j
 -0.06460488+0.05088478j  0.00018227+0.06738j    -0.04732586-0.03858394j
  0.05418138-0.02990856j  0.01890594-0.00596687j] Norm= 0.18680421761750834
Homogeneous solution from LCHS : [-0.05605665+0.00883657j -0.02036921-0.0526817j   0.09148876-0.00032831j
 -0.061482  +0.04958058j  0.00180499+0.06495725j -0.0464175 -0.03942093j
  0.05268995-0.02966901j  0.01956579-0.00648528j] Norm= 0.18174687981381626
Homogeneous solution error: 0.0067795127088803525 Relative error: 0.03629207517552826

Normalized
Homogeneous solution from scipy: [-0.31881796+0.04224231j -0.11137225-0.27939868j  0.50430076+0.00218107j
 -0.34584273+0.27239631j  0.00097574+0.36069851j -0.25334471-0.20654747j
  0.29004365-0.16010645j  0.10120722-0.03194183j] Norm= 1.0
Homogeneous solution from LCHS : [-0.30843255+0.0486202j  -0.11207459-0.28986305j  0.50338561-0.00180639j
 -0.33828365+0.272

## Inhomogeneous Solution


Consider we have extra b(t) term in the ODE
$$
\frac{\mathrm{d} u(t)}{\mathrm{d} t} = -A u(t) + b(t),\quad u(0) = u_0
$$

In [19]:
def trunc_K_inho(beta, epsilon, bnorm, h1):
    ## Lemma 12 (Eq. (73) ) in [1]
    temp = (np.log(1+bnorm/epsilon))**(1/beta)
    return np.ceil(temp/h1)*h1

def norm_lambda(A):
    ## \Lambda = \sup_{p \geq 0, t\in [0,T]} \|A^{(p)}\|^{1/(p+1)}
    ## A is time independent, so just 0th order time derivative
    return nl.norm(A, ord=2)

def norm_Xi(t0, tT, p, func_btp=bt):
    ## Use func_bt for the function output time deritatives of b(t)
    ## \Xi = \sup_{p \geq 0, t\in [0,T]} \|b^{(p)}\|^{1/(p+1)}
    time_slices = 100
    time_points = np.linspace(t0, tT, time_slices)
    max_norm = 0
    for time_point in time_points:
        norm_val = nl.norm(func_btp(time_point), ord=2)
        temp = np.power( norm_val, 1/(p+1) ) if np.abs(norm_val-1e-8) >0 else 0
        if temp > max_norm:
            max_norm = temp
    return max_norm

def norm_bL1(t0, tT, func_bt=bt):
    ## \int_{s\in [0,T]} \|b(s)\| ds
    time_slices = 100
    samp_points, weights = np.polynomial.legendre.leggauss(time_slices)
    time_step = (tT-t0)
    weights = 0.5*time_step*weights
    samp_points = 0.5*time_step*samp_points + (t0 + 0.5*time_step)
    temp_sum = 0
    for i in range(len(samp_points)):
        temp_sum += weights[i]*nl.norm(func_bt(samp_points[i]),ord=2)
    return temp_sum

def scipy_int(t0, tT, int_func=bt):
    ## an alternative for norm_bL1, just for checking
    return scipy.integrate.quad(lambda x: nl.norm(int_func(x),ord=2), t0, tT)

def step_size_h2(K, Lam, Xi, tT):
    ## Lemma 12 (Eq. (73) ) in [1]
    ## h_2 = \frac{1}{eK(\Lambda+\Xi)}
    ## Note, the return is <= \lceil T/h_2  \rceil to make the number of slices integer
    theo_h2 = 1/(np.exp(1)*K*(Lam + Xi))
    slice_T = np.ceil(tT/theo_h2)
    return tT/slice_T

def n_node_Q2_inho(epsilon, tT, Lam, Xi, cnorm=1):
    ## Combine Eq. (234) and (235) in [1]
    return int(np.ceil(cnorm*1/(np.log(4))*np.log( 2*np.exp(1)*tT*(Lam+Xi)/epsilon )))


bL1_exp = norm_bL1(0, T, func_bt=bt)
bLi_scipy_exp = scipy_int(0, T, int_func=bt)
bL1d1_exp = norm_bL1(0, T, func_bt=bt_d1)
h1_exp = step_size_h1(T, L_exp)
K_inho_exp = trunc_K_inho(beta, epsilon, bL1_exp, h1_exp)
Lam_exp = norm_lambda(A)
Xi_exp = norm_Xi(0, T, 0, func_btp=bt)
# Xi_exp = norm_Xi(0, T, 1, func_btp=bt_d1)
h2_exp = step_size_h2(K_inho_exp, Lam_exp, Xi_exp, T)
Q1_exp = n_node_Q(beta, epsilon, K_inho_exp)
Q2_exp = n_node_Q2_inho(epsilon, T, Lam_exp, Xi_exp, cnorm=1)

print("||b||_L1=", bL1_exp, "from scipy:",bLi_scipy_exp)
print("||b'||_1=", bL1d1_exp)
print("K for inhomogeneous case=", K_inho_exp)
print("Lambda=", Lam_exp)
print("Xi=", Xi_exp)
print("h2=", h2_exp)
print("Q1 =", Q1_exp)
print("Q2 =", Q2_exp)

||b||_L1= 3.338149441972587 from scipy: (3.3381494419725857, 3.7060903701175826e-14)
||b'||_1= 4.011234224026317
K for inhomogeneous case= 7.096166251616453
Lambda= 7.364612689144848
Xi= 4.726520919238589
h2= 0.004273504273504274
Q1 = 6
Q2 = 7


In [20]:
def class_LCHS_tips(A, u0, tT, beta, epsilon, func_bt=bt, func_normbt=bt_d1, verbose=False): 
    ## Lemma 12 in [1]
    L,H = generate_LH(A)
    bL1 = norm_bL1(0, T, func_bt=func_bt)
    h1 = step_size_h1(tT, L)  ## step size h1 for [-K, K], (65) in [1]
    K = 2*trunc_K_inho(beta, epsilon, bL1, h1)
    Lam = norm_lambda(A)
    Xi = norm_Xi(0, T, 0, func_btp=func_bt)
    # Xi = norm_Xi(0, T, 1, func_btp=func_normbt)
    h2 = step_size_h2(K, Lam, Xi, tT)
    Q1 = n_node_Q(beta, epsilon, K)
    Q2 = n_node_Q2_inho(epsilon, T, Lam, Xi, cnorm=1)

    kh1 = int(K/h1)
    Th2 = int(T/h2)
    M = int(2*kh1*Q1)
    Mp = int(Th2*Q2)

    if verbose:
        print("Preset parameters T = ", tT, "beta = ", beta, "epsilon = ", epsilon)
        print("Truncation range [-K,K] K =", K)
        print("Step size h1 =", h1)
        print("Step size h2 =", h2)
        print("Number of nodes in [mh_1, (m+1)h_1] Q1 =", Q1)
        print("Number of nodes in [mh_2, (m+1)h_2] Q2 =", Q2)
        print("Lambda=", Lam)
        print("Xi=", Xi)
        print("Total number of nodes M =", M)
        print("Total number of nodes M' =", Mp)
        print("Total number of nodes M*M' =", M*Mp)

    res_mat = np.zeros(u0.shape, dtype=complex)
    c_sum = 0 ## compute ||c||_1

    mh1_range = []
    mh2_range = []
    
    for munshit in range(2*kh1):
        m = -kh1+munshit ## shift to start from -K/h1
        kqms, wqs = gauss_quadrature(m*h1,h1,Q1) ## Gaussian quadrature points and weights in [mh_1, (m+1)h_1]
        mh1_range.append(m*h1)
        for qi in range(Q1):
            c1qm = wqs[qi]*gk(beta, kqms[qi], cb_func=cbeta)
            c_sum += np.abs(c1qm)
            for m2 in range(Th2):
                sqms, w2qs = gauss_quadrature(m2*h2,h2,Q2)
                mh2_range.append(m2*h2)
                for qi2 in range(Q2):
                    sqm = sqms[qi2]
                    bsqm = np.array(func_bt(sqm))
                    c2qm = w2qs[qi2]*nl.norm(bsqm, ord=2)
                    basqm_norm = bsqm/nl.norm(bsqm, ord=2)
                    print("Progress: ", f"{munshit}/{2*kh1}",f"{m2}/{Th2}", end="\r")
                    res_mat += c2qm*c1qm* (utsk(tT,sqms[qi2],kqms[qi],L,H).dot(basqm_norm))  ## This should be quantum
    if verbose:
        print()
        print("||c||_1 =", c_sum)
    return res_mat, mh1_range, mh2_range


uT_inho, rang1, rang2 = class_LCHS_tips(A, u0, T, beta, epsilon, verbose=True)
uT_inho = np.array(uT_inho).reshape(-1)
uT_inho_norm = uT_inho/nl.norm(uT_inho,ord=2)
print("particular solution u(T)=", uT_inho)
print("particular solution u(T)/||u(T)||=", uT_inho_norm)

Preset parameters T =  1 beta =  0.9 epsilon =  0.01
Truncation range [-K,K] K = 14.192332503232906
Step size h1 = 0.060137002132342826
Step size h2 = 0.0021413276231263384
Number of nodes in [mh_1, (m+1)h_1] Q1 = 6
Number of nodes in [mh_2, (m+1)h_2] Q2 = 7
Lambda= 7.364612689144848
Xi= 4.726520919238589
Total number of nodes M = 2832
Total number of nodes M' = 3269
Total number of nodes M*M' = 9257808
Progress:  471/472 466/467
||c||_1 = 1.9245385376887225
particular solution u(T)= [ 0.17666921+0.53637593j  0.27178857-0.14776146j -0.40495624+0.58272976j
  0.46320921+0.18214014j -0.00924397-0.21582075j  0.18797219-1.0000327j
 -0.2103238 -0.14147541j  0.23915522-0.09041355j]
particular solution u(T)/||u(T)||= [ 0.11457544+0.34785637j  0.17626329-0.09582788j -0.26262664+0.37791826j
  0.30040549+0.11812351j -0.005995  -0.13996643j  0.12190577-0.64855212j
 -0.13640148-0.09175118j  0.15509956-0.05863598j]


### Compare with Scipy Implementation

Seems like the choice of $K$ has better impact on the solution accuracy than $Q_2$, as it quadratically increase the total discritization slices

In [21]:
def ode_func_inho(t,u):
    res = np.array(-A.dot(u)+bt(t).reshape(-1))[0]
    return res
    # return np.array(-(L_exp+1j*H_exp).real.dot(u)+bt(t).reshape(-1))[0]

spi_sol_inho = spi.solve_ivp(ode_func_inho, [0,T],np.array(u0.reshape(-1))[0], method='RK45')
spi_sol_ps = spi_sol_inho.y[:,-1] - spi_sol_ho.y[:,-1] ## subtract homogeneous solution
spi_sol_ps_norm = spi_sol_ps/nl.norm(spi_sol_ps,ord=2)

if nl.norm(uT_inho.imag,ord=2) < 1e-12:
    uT_inho = uT_inho.real

if nl.norm(uT_inho_norm.imag,ord=2) < 1e-12:
    uT_inho_norm = uT_inho_norm.real

inho_sol_err = nl.norm(uT_inho - spi_sol_ps,ord=2)
inho_sol_err_norm = nl.norm(uT_inho_norm - spi_sol_ps_norm,ord=2)

print()
print("epsilon:", epsilon)
print("\nUn-normalized")
print("Inhomogeneous solution from scipy:", spi_sol_ps, "Norm=", nl.norm(spi_sol_ps,ord=2))
print("Inhomogeneous solution from LCHS :", uT_inho, "Norm=", nl.norm(uT_inho,ord=2))
print("Inhomogeneous solution error:", inho_sol_err, "Relative error:", inho_sol_err/nl.norm(spi_sol_ps,ord=2))
print("\nNormalized")
print("Inhomogeneous solution from scipy:", spi_sol_ps_norm, "Norm=", nl.norm(spi_sol_ps_norm,ord=2))
print("Inhomogeneous solution from LCHS :", uT_inho_norm, "Norm=", nl.norm(uT_inho_norm,ord=2))
print("Inhomogeneous solution error:", inho_sol_err_norm, "Relative error:", inho_sol_err_norm/nl.norm(spi_sol_ps_norm,ord=2))


epsilon: 0.01

Un-normalized
Inhomogeneous solution from scipy: [ 0.17827178+0.53683849j  0.27198706-0.14805453j -0.40621576+0.58554185j
  0.46426809+0.18139484j -0.01048231-0.21673704j  0.18797721-1.00163703j
 -0.21034567-0.14093346j  0.23970109-0.0904041j ] Norm= 1.5451948090573018
Inhomogeneous solution from LCHS : [ 0.17666921+0.53637593j  0.27178857-0.14776146j -0.40495624+0.58272976j
  0.46320921+0.18214014j -0.00924397-0.21582075j  0.18797219-1.0000327j
 -0.2103238 -0.14147541j  0.23915522-0.09041355j] Norm= 1.541946548175626
Inhomogeneous solution error: 0.004429184089703485 Relative error: 0.002866424391113285

Normalized
Inhomogeneous solution from scipy: [ 0.11537172+0.34742447j  0.17602121-0.0958161j  -0.26288967+0.37894371j
  0.30045926+0.11739286j -0.00678381-0.14026519j  0.12165276-0.64822702j
 -0.1361289 -0.09120757j  0.15512678-0.0585066j ] Norm= 1.0
Inhomogeneous solution from LCHS : [ 0.11457544+0.34785637j  0.17626329-0.09582788j -0.26262664+0.37791826j
  0.3004054

## Full Solution

In [24]:
uT_full = uT + uT_inho
uT_full_norm = uT_full/nl.norm(uT_full,ord=2)

spi_full_sol = spi_sol_inho.y[:,-1]
spi_full_sol_norm = spi_full_sol/nl.norm(spi_full_sol,ord=2)

full_sol_err = nl.norm(uT_full - spi_full_sol,ord=2)
full_sol_err_norm = nl.norm(uT_full_norm - spi_full_sol_norm,ord=2)


print("Full Solution")
print("epsilon:", epsilon)
print("\n------------------------------------------------------------------------")
print("\nUn-normalized, as in the paper")
print("Full solution from scipy:", spi_full_sol, "  Norm=", nl.norm(spi_full_sol,ord=2))
print("Full solution from LCHS :", uT_full, "  Norm=", nl.norm(uT_full,ord=2))
print("\nFull solution error             :", full_sol_err, "    Relative error:", full_sol_err/nl.norm(spi_full_sol,ord=2))
print("  - Homogeneous solution error  :", uT_err, "   Relative error:", uT_err/nl.norm(spi_uT_ho,ord=2))
print("  - Inhomogeneous solution error:", inho_sol_err, "    Relative error:", inho_sol_err/nl.norm(spi_sol_ps,ord=2))


print("\n------------------------------------------------------------------------")
print("\nNormalized Solution State")
print("Full solution from scipy:", spi_full_sol_norm, "  Norm=", nl.norm(spi_full_sol_norm,ord=2))
print("Full solution from LCHS :", uT_full_norm, "  Norm=", nl.norm(uT_full_norm,ord=2))
print("\nFull solution error             :", full_sol_err_norm, "    Relative error:", full_sol_err_norm/nl.norm(spi_full_sol_norm,ord=2))
print("  - Homogeneous solution error  :", uT_err_norm, "     Relative error:", uT_err_norm/nl.norm(spi_uT_ho_norm,ord=2))
print("  - Inhomogeneous solution error:", inho_sol_err_norm, "   Relative error:", inho_sol_err_norm/nl.norm(spi_sol_ps_norm,ord=2))

Full Solution
epsilon: 0.01

------------------------------------------------------------------------

Un-normalized, as in the paper
Full solution from scipy: [ 0.11871524+0.54472953j  0.25118225-0.20024739j -0.31201025+0.58594928j
  0.39966321+0.23227962j -0.01030004-0.14935704j  0.14065135-1.04022096j
 -0.15616429-0.17084202j  0.25860703-0.09637097j]   Norm= 1.523936315314447
Full solution from LCHS : [ 0.12061255+0.5452125j   0.25141936-0.20044317j -0.31346747+0.58240145j
  0.40172722+0.23172071j -0.00743898-0.1508635j   0.14155469-1.03945363j
 -0.15763384-0.17114442j  0.25872101-0.09689883j]   Norm= 1.5236493251972403

Full solution error             : 0.006132959788895681     Relative error: 0.004024420001849102
  - Homogeneous solution error  : 0.0067795127088803525    Relative error: 0.03629207517552826
  - Inhomogeneous solution error: 0.004429184089703485     Relative error: 0.002866424391113285

------------------------------------------------------------------------

Normal